In [9]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json


getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended, process_datesIT

In [10]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [11]:
independents = urls[[name for name in urls.index if name[0] == 'G']]
dependents = urls[[name for name in urls.index if name[0] != 'G' and name[0] != 'U']]
#dependents2 = urls[[name for name in urls.index if name[0] in 'TSEIFXC']]
unused = urls[[name for name in urls.index if name[0] == 'U']]

In [12]:
! rm ../../Finances.db
! rm schema.sql

In [13]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [14]:
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()
schemas = open("schema.sql", "a+")

In [15]:
%%time
for name, url in independents.iteritems():
    
    tab = get_appended(url,apikey,name)
    a = pd.io.sql.get_schema(tab,name)
    c.execute('drop table if exists '+ name)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(name,con=conn,index=False,if_exists='append')

CPU times: user 1.02 s, sys: 105 ms, total: 1.12 s
Wall time: 19.6 s


In [16]:
###FULL lists for full set processing
stocks = list(pd.io.sql.read_sql('select symbol from G1_CV_SYMBOLS;',con=conn).symbol)
etfs  = list(pd.io.sql.read_sql('select symbol from G1_ETF_SYMBOLS;',con=conn).symbol)
etfs2  = list(pd.io.sql.read_sql('select symbol from G1_CV_ETFS;',con=conn).symbol)
indexes = list(pd.io.sql.read_sql('select symbol from G1_MI_MAJORINDEXES;',con=conn).symbol)
ciks = [str(c) for c in list(pd.io.sql.read_sql('select cik from G1_INST_CIKLIST',con=conn).cik)]
funds = list(pd.io.sql.read_sql('select symbol from G1_MF_SYMBOLS;',con=conn).symbol)
commods = list(pd.io.sql.read_sql('select symbol from G1_COM_SYMBOLS;',con=conn).symbol)
##Light cleanup, nothing perfect –– could set this up using sql. 

#merge etf lists
etfs_c = list(set(etfs).union(set(etfs2)))
#leave stocks only 
stocks_c = list(set(stocks) - set(etfs + indexes + ciks + funds + commods))

#dictionary – nothing commodity, crypto, etc. specific
d = {'H':stocks_c + etfs_c + commods + funds,'T':stocks_c + etfs_c + commods,'S':stocks_c,'E':etfs_c,'C':ciks,'I':indexes}

In [9]:
#create smaller samples of stocks & etfs for 
sp500stocks = list(pd.io.sql.read_sql('select symbol from G1_MI_SP500LIST;',con=conn).symbol)

In [10]:
profiles = get_appended(dependents.iloc[0],apikey,dependents.index[0],tickers=d.get('T'))

In [13]:
for name, url in dependents.iteritems():
    tab = get_appended(url,apikey,name,tickers=d.get(name[0]))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27631 entries, 0 to 29916
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symbol             27631 non-null  object 
 1   price              27631 non-null  float64
 2   beta               27631 non-null  float64
 3   volAvg             27391 non-null  float64
 4   mktCap             27625 non-null  float64
 5   lastDiv            27325 non-null  float64
 6   range              27484 non-null  object 
 7   changes            27631 non-null  float64
 8   companyName        27627 non-null  object 
 9   currency           27631 non-null  object 
 10  cik                10101 non-null  object 
 11  isin               19002 non-null  object 
 12  cusip              18936 non-null  object 
 13  exchange           27547 non-null  object 
 14  exchangeShortName  27631 non-null  object 
 15  industry           26793 non-null  object 
 16  website            267

In [85]:
url1 = 'https://financialmodelingprep.com/api/v3/historical-price-full/ ?apikey='
url2 = url1.replace(' ',','.join(stocks[0:4])) + apikey
#url2 = url1.replace(' ','FB') + apikey

In [86]:
pd.read_json(url2)

,historicalStockList
0,"{'symbol': 'CMCSA', 'historical': [{'date': '2..."
1,"{'symbol': 'KMI', 'historical': [{'date': '202..."
2,"{'symbol': 'INTC', 'historical': [{'date': '20..."
3,"{'symbol': 'SPY', 'historical': [{'date': '202..."


""


In [13]:
# #etfs roughly 1:5 stocks.. get top 100 etfs
etfsval = pd.DataFrame()
for i in range(iterations):
    start = i * 1000
    stop = (i+1) * 1000 -1
    print(start,stop)
    etfsval = etfsval.append(pd.read_json("https://financialmodelingprep.com/api/v3/profile/"+','.join(etfs_c[start:stop])+"?apikey="+apikey))

0 999
1000 1999
2000 2999


In [14]:
etfsval.sort_values(by='volAvg',ascending=False).iloc[:100].symbol

2      UVXY
0       SPY
9      SQQQ
4       VXX
0       EEM
       ... 
13      KBE
93     GUSH
28      VEU
702    ARKF
19      ITB
Name: symbol, Length: 100, dtype: object

In [50]:
url = 'https://financialmodelingprep.com/api/v3/historical-price-full/'+','.join(etfs_c[89:94])+'?timeseries=100000000&apikey='+apikey

In [51]:
pd.read_json(url)

,historicalStockList
0,"{'symbol': 'PSCI', 'historical': [{'date': '20..."
1,"{'symbol': 'KOKU', 'historical': [{'date': '20..."
2,"{'symbol': 'COGO.L', 'historical': [{'date': '..."
3,"{'symbol': 'VCN.TO', 'historical': [{'date': '..."
4,"{'symbol': 'IFRA', 'historical': [{'date': '20..."


'https://financialmodelingprep.com/api/v3/historical-price-full/AMLP,OPER,VTHR,CRPX.L,HVU.TO,ZBZX,SCHX,YOLO,DRN,NRGU,INTL.L,FCG,PWS,NULV,EWA,VFV.TO,HFU.TO,LUXE,GSID,SMDY,RWUI,PSCE,ESGL.L,XEI.TO,HGC.TO,SIXH,GOVZ,DFNV,HAU.TO,RIG.TO,ESNG,CHNA,FQAL,OVB,MSUS,EWO,EQL,MNA,XCH.TO,ZLC.TO,FRDM,VALQ,BSJL,HYLB,SMB,SGDJ,XOP,XCG.TO,IWM,IBBJ,XIC.TO,XAR,XLB.TO,CIZ,FNGZ,URE,FCAL,JIG,FSZ,NUBD,ESEB,EMFM,FLV,DUAL,HFR.TO,GNRX,IBDU,PBUS,QGRO,FST.TO,FTA,JUST,TSCPX,SEIX,UITB,SJNK,DWX,JPUS,JNK,EMXC,EQLT,FPEI,JHMT,MMTM,IPAY,XHS,LRNZ,EDZ,LOWC,DXJS,FLGV,TFLO,BAUG,KCE,JHMC,FEX,FAD,SYG,THE.TO,UCC,PAMC,BUFR,XSHQ,AZBA,AVUS,RSGL.L,VXM.TO,SPVU,FGO.TO,PSK,PUTW,PAK,LQDH,RWSL,SIXL,QCD.TO,MDYV,TGRW,EWZ,IUSV,DEFA,FTXL,KCCB,RFEU,XTR.TO,RNDM,QMV.TO,LVHI,VFMV,DFND,SHY,HSL.TO,ZJK.TO,MLPR,FLFR,IYC,ROSC,IXJ,HEU.TO,ROUS,FMHI,ZMI.TO,DGRW,DIA,RWR,FTGC,XMLV,CNXT,CHIH,FNGU,PMOM,DVY,GERM,OMFS,IYM,FEM,ZLI.TO,EJUL,BSCM,SUSB,ZPAY.TO,XBM.TO,BSMT,DYHG,SCHP,CDL,PLC,EXI,QQQ,PICK,KORU,BSCS,CMBS,DWAT,TMDV,SPXL,SPEM,HYEM,AUGZ,ROOF,ABEQ,QINT,DWAW

In [117]:
'https://financialmodelingprep.com/api/v3/stock_news?tickers=AAPL,GOOGL,SPY&limit=10000000&apikey=600e501fe324b2f1cbb2aec5b390b6db'

'https://financialmodelingprep.com/api/v3/stock_news?tickers=AAPL,GOOGL,SPY&limit=10000000&apikey=600e501fe324b2f1cbb2aec5b390b6db'

In [116]:
"https://financialmodelingprep.com/api/v3/stock_news?tickers="+','.join(etfs_c[0:4])+"&limit=100000?apikey="+apikey

'https://financialmodelingprep.com/api/v3/stock_news?tickers=HEEM,MMLG,EJUL,FSF.TO&limit=100000?apikey=600e501fe324b2f1cbb2aec5b390b6db'

In [11]:
#nothing commodity, crypto, etc. specific
dsub = {'T':stocks_c + etfs_c + commods,'S':stocks_c,'E':etfs_c,'F':funds,'I':indexes}

In [12]:
def get_subset():
    for name, 

SyntaxError: invalid syntax (<ipython-input-12-4f680e375e9b>, line 2)

In [61]:
stocks2 = stocks.copy()

In [65]:
tickers = ','.join(etfs2[:1000])

In [66]:
url = 'https://financialmodelingprep.com/api/v3/profile/ ?apikey='.replace(' ',tickers) + apikey

In [67]:
test = pd.read_json(url)

In [68]:
test

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading
0,SPY,384.760,0.0,65782473,353125827000,5.69100,218.26-394.17,2.430,SPDR S&P 500,USD,...,None,None,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,1993-01-22,False,True,True
1,GDX,31.165,0.0,23635152,9892389525,0.19000,16.18-45.78,-1.165,VanEck Vectors Gold Miners,USD,...,None,None,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,2006-05-16,False,True,True
2,EEM,53.960,0.0,41095726,40599503000,0.74900,30.1-58.29,-0.500,iShares MSCI Emerging Index Fund,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,2003-04-07,False,True,True
3,EFA,74.420,0.0,20674478,69835727000,1.55200,45.73-77.17,-0.650,iShares MSCI EAFE,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,2001-08-14,False,True,True
4,QQQ,318.770,0.0,32120832,125308486000,1.73624,164.93-338.19,5.940,PowerShares QQQ Trust Series 1,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,1999-10-03,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,FTA,59.580,0.0,63185,1199072107,1.11400,31.06-60.97,0.030,First Trust Large Cap Value AlphaDEX Fund,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,2007-05-08,False,True,True
996,FTAG,29.270,0.0,845,4525965,0.45400,13.91-29.61,-0.010,First Trust Indxx Global Agriculture ETF,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,None,False,True,True
997,FTGC,21.330,0.0,348591,172918679,0.15400,13.05-21.99,-0.370,First Trust Global Tactical Commodity Strategy...,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,2013-10-23,True,True,True
998,FTHI,20.515,0.0,10377,63059970,0.96000,10.63-21.61,-0.135,First Trust BuyWrite Income ETF,USD,...,,,NaN,NaN,0.0,https://financialmodelingprep.com/image-stock/...,2014-01-06,False,True,True


In [19]:
def get_dependents():
    global dependents
    for name, url in dependents.iteritems():
        symbols = d.get(name[0])
        tab = get_appended(url,apikey,symbols,json=name[1],verbose=True)
        a = pd.io.sql.get_schema(tab,name)
        c.execute('drop table if exists '+ name)
        c.execute(a)
        tosql = a +';\n\n'
        schemas.write(tosql)
        tab.to_sql(name,con=conn,index=False,if_exists='append')

KeyboardInterrupt: 

In [23]:
url = 'https://financialmodelingprep.com/api/v3/income-statement/AAPL?limit=120&apikey=600e501fe324b2f1cbb2aec5b390b6db'

In [27]:
a = pd.read_json('https://financialmodelingprep.com/api/v3/historical-discounted-cash-flow/AAPL?period=quarter&apikey=600e501fe324b2f1cbb2aec5b390b6db')

In [31]:
pd.DataFrame(a.iloc[0,1])

,date,Stock Price,DCF
0,2020-09-26,115.050003,117.240859
1,2020-06-27,94.809998,96.294226
2,2020-03-28,70.792503,72.481782
3,2019-12-28,77.237503,78.894946
4,2019-09-28,62.262501,64.396395
5,2019-06-29,52.419998,54.430319
6,2019-03-30,51.152500,52.614161
7,2018-12-29,39.075001,40.908080
8,2018-09-29,53.060001,53.943303
9,2018-06-30,47.477501,50.216961
